In [1]:
from sqlalchemy import create_engine
import sqlalchemy
import pandas as pd
from sqlalchemy import inspect
from dotenv import load_dotenv
import os
import sys

sys.path.insert(0,'..')

In [2]:
dotenv_path = os.path.join('..', '.env')
load_dotenv(dotenv_path)

SQL_CONNECTOR = os.getenv('SQL_CONNECTOR')

In [4]:
SQL_CONNECTOR

'mysql+mysqldb://team20:te@mZoP@ss@giniewicz.it:3306/team20?charset=utf8'

In [22]:
engine = create_engine(SQL_CONNECTOR, encoding='utf8')
insp = inspect(engine)
insp.get_schema_names()

['information_schema', 'team20']

In [3]:
tables = insp.get_table_names()

In [23]:
pracownicy = pd.read_csv('INSERT/pracownicy.csv', index_col='Unnamed: 0', encoding='utf8', parse_dates=['Data zatrudnienia'])
pracownicy['Data zatrudnienia'] = pd.to_datetime(pracownicy['Data zatrudnienia'], format='%d-%m-%Y')
sprzęt = pd.read_csv('INSERT/sprzęt.csv')
wizyty = pd.read_csv('INSERT/wizyty.csv', index_col='Unnamed: 0', parse_dates=['Data'])
wlasciciele = pd.read_csv('INSERT/właściciele.csv', index_col='Unnamed: 0', dtype={'PESEL':str})
zwierzeta = pd.read_csv('INSERT/zwierzeta.csv', index_col='Unnamed: 0', parse_dates=['Data rejestracji', 'Data urodzenia'])
transakcje = pd.read_csv('INSERT/transakcje.csv', parse_dates=['Data'])

In [50]:
transakcje.loc[~transakcje['id_wizyty jeśli wizyta'].isna(), 'id_wizyty jeśli wizyta'] = transakcje.loc[~transakcje['id_wizyty jeśli wizyta'].isna(), 'id_wizyty jeśli wizyta'].astype(int)
transakcje.columns = ['id_transakcji', 'Data', 'Rodzaj transakcji', 'Kwota',
       'id_wizyty jeśli wizyta']

In [5]:
pracownicy.to_sql('Pracownicy', con=engine, if_exists='replace', index=False)

In [6]:
wizyty.columns = ['id_wizyty', 'Data', 'Godzina', 'id_pracownika', 'Czas trwania wizyty',
       'Rodzaj wizyty', 'id_zwierzęcia']

In [7]:
wizyty['Godzina'] = pd.to_datetime(wizyty['Godzina'], format='%H:%M:%S').dt.time
wizyty['Czas trwania wizyty'] = pd.to_datetime(wizyty['Czas trwania wizyty'], format='%H:%M').dt.time

In [24]:
wizyty['id_wizyty'] = wizyty['id_wizyty']  + 1

wlasciciele['id_właściciela'] = wlasciciele['id_właściciela'] + 1

zwierzeta['id_zwierzęcia'] = zwierzeta['id_zwierzęcia'] + 1

In [169]:
wizyty.to_sql('Wizyty', con=engine, if_exists='append', index=False)

In [26]:
wlasciciele.to_sql('Właściciele', con=engine, if_exists='append', index=False)

In [9]:
zwierzeta.to_sql('Zwierzęta', con=engine, if_exists='append', index=False)

In [7]:
sprzęt['Wartość'] = sprzęt['Wartość'].replace(r'\xa0', '', regex=True)
sprzęt['Wartość'] = sprzęt['Wartość'].astype(int)

In [11]:
sprzęt.to_sql('Sprzęt', con=engine, if_exists='append', index=False)

In [52]:
transakcje.to_sql('Transakcje', con=engine, if_exists='append', index=False)